In [1]:
from mrf_util import *

In [5]:
# create_local_patches_sanity_check_1 
with tf.Session() as sess:
    batch, height, width, feature = 1, 3, 3, 1
    patch_size = 3
    generated_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    local_patches = create_local_patches(generated_layer, patch_size)
    
    init_generated_layer = np.array([[[[1], [2], [3]], [[4], [5], [6]], [[7], [8], [9]]]])
    
    feed_dict = {generated_layer: init_generated_layer}
    
    expected_output = np.array([[[[1, 2, 3, 4, 5, 6, 7, 8, 9]]]])
    print(np.all((local_patches.eval(feed_dict) - expected_output) == 0))

True


In [6]:
# create_local_patches_sanity_check_2
with tf.Session() as sess:
    batch, height, width, feature = 1, 3, 3, 1
    patch_size = 2
    generated_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    local_patches = create_local_patches(generated_layer, patch_size)
    
    init_generated_layer = np.array([[[[1], [2], [3]], [[4], [5], [6]], [[7], [8], [9]]]])
    
    feed_dict = {generated_layer: init_generated_layer}
    
    expected_output = np.array([[[[1, 2, 4, 5], [2, 3, 5, 6]], [[4, 5, 7, 8], [5, 6, 8, 9]]]])
    print(np.all((local_patches.eval(feed_dict) - expected_output) == 0))

True


In [8]:
# create_local_patches_sanity_check_3
with tf.Session() as sess:
    batch, height, width, feature = 1, 3, 3, 2
    patch_size = 2
    generated_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    local_patches = create_local_patches(generated_layer, patch_size)
    
    init_generated_layer = np.array(
                [[[[1, 1], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]]])    
    feed_dict = {generated_layer: init_generated_layer}
    
    expected_output = np.array([[[[1, 1, 2, 2, 4, 4, 5, 5], [2, 2, 3, 3, 5, 5, 6, 6]],
                                         [[4, 4, 5, 5, 7, 7, 8, 8], [5, 5, 6, 6, 8, 8, 9, 9]]]])
    print(expected_output.shape)
    print(np.all((local_patches.eval(feed_dict) - expected_output) == 0))

(1, 2, 2, 8)
True


In [3]:
# patch_matching_sanity_check_1
with tf.Session() as sess:
    batch, height, width, feature = 1, 3, 3, 1
    patch_size = 2
    generated_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    generated_layer_patches = create_local_patches(generated_layer, patch_size)

    style_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    style_layer_patches = create_local_patches(style_layer, patch_size)

    actual_output = patch_matching(generated_layer_patches, style_layer_patches, patch_size)

    init_generated_layer = np.array([[[[1], [2], [3]], [[4], [5], [6]], [[7], [8], [9]]]])
    feed_dict = {generated_layer: init_generated_layer, style_layer: init_generated_layer}
    expected_output = np.array([[[[1, 2, 4, 5], [2, 3, 5, 6]], [[4, 5, 7, 8], [5, 6, 8, 9]]]])
    print(actual_output.eval(feed_dict))
    print(np.all((actual_output.eval(feed_dict) - expected_output) < 1e-3))

[[[[ 1.  2.  4.  5.]
   [ 2.  3.  5.  6.]]

  [[ 4.  5.  7.  8.]
   [ 5.  6.  8.  9.]]]]
True


In [3]:
# patch_matching_sanity_check_2
with tf.Session() as sess:
    batch, height, width, feature = 2, 3, 3, 2
    patch_size = 2
    generated_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    generated_layer_patches = create_local_patches(generated_layer, patch_size)

    style_layer = tf.placeholder(tf.float32, shape=(1, height, width, feature))
    style_layer_patches = create_local_patches(style_layer, patch_size)

#     normalized_style_layer_patches = tf.nn.l2_normalize(style_layer_patches, dim=[3])
    actual_output = patch_matching(generated_layer_patches, style_layer_patches, patch_size)

    init_generated_layer = np.array(
    [[[[1, 1], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]],
     [[[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]], [[7, 7], [8, 8], [9, 9]]]])
    init_style_layer = np.array(
        [[[[1, 1], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]]])
    
    feed_dict = {generated_layer: init_generated_layer, style_layer: init_style_layer}
    expected_output = np.array([[[[1, 1, 2, 2, 4, 4, 5, 5], [2, 2, 3, 3, 5, 5, 6, 6]],
                             [[4, 4, 5, 5, 7, 7, 8, 8], [5, 5, 6, 6, 8, 8, 9, 9]]],
                            [[[4, 4, 5, 5, 7, 7, 8, 8], [5, 5, 6, 6, 8, 8, 9, 9]],
                             [[5, 5, 6, 6, 8, 8, 9, 9], [5, 5, 6, 6, 8, 8, 9, 9]]]])
#     expected_output = expected_output / np.expand_dims(np.linalg.norm(expected_output, axis=(3)), axis=3)
    print(np.all((actual_output.eval(feed_dict) - expected_output) < 1e-3))
    print(actual_output.eval(feed_dict))
    print(actual_output.eval(feed_dict).shape)

True
[[[[ 1.  1.  2.  2.  4.  4.  5.  5.]
   [ 2.  2.  3.  3.  5.  5.  6.  6.]]

  [[ 4.  4.  5.  5.  7.  7.  8.  8.]
   [ 5.  5.  6.  6.  8.  8.  9.  9.]]]


 [[[ 4.  4.  5.  5.  7.  7.  8.  8.]
   [ 5.  5.  6.  6.  8.  8.  9.  9.]]

  [[ 5.  5.  6.  6.  8.  8.  9.  9.]
   [ 5.  5.  6.  6.  8.  8.  9.  9.]]]]
(2, 2, 2, 8)


In [2]:
# mrf_loss => 나중에 프린트할때 배치 사이즈로 나눠주기
with tf.Session() as sess:
    batch, height, width, feature = 2, 3, 3, 2
    patch_size = 2
    generated_layer = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    generated_layer_patches = create_local_patches(generated_layer, patch_size)

    style_layer = tf.placeholder(tf.float32, shape=(1, height, width, feature))
    style_layer_patches = create_local_patches(style_layer, patch_size)

    normalized_style_layer_patches = tf.nn.l2_normalize(style_layer_patches, dim=[3])
    output = mrf_loss(style_layer, generated_layer, patch_size=patch_size)

    init_generated_layer = np.array(
    [
        [[[1, 1], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]],
        [[[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]], [[7, 7], [8, 8], [9, 9]]]
    ])
    init_style_layer = np.array(
        [[[[1, 1], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]]])
    
    feed_dict = {generated_layer: init_generated_layer, style_layer: init_style_layer}
    expected_output = (2.0 ** 2 * 4   +   1.0 ** 2 * 4   +   3.0 ** 2 * 4) / patch_size ** 2 / height * width * feature
    print(output.eval(feed_dict))
    print(np.all((output.eval(feed_dict) - expected_output) < 1e-3))

0.777778
True


In [17]:
# study of loss [divide by what?]
# answer is := height * width * dimension
with tf.Session() as sess:
    batch, height, width, feature = 1, 3, 3, 3
    a = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    b = tf.placeholder(tf.float32, shape=(batch, height, width, feature))
    loss = tf.losses.mean_squared_error(a, b)

#     init_a = np.array([[[[1, 1], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]]])
#     init_b = np.array([[[[3, 2], [2, 2], [3, 3]], [[4, 4], [5, 5], [6, 6]], [[7, 7], [8, 8], [9, 9]]]])
    
    init_a = np.array([[[[1, 1, 1], [2, 2, 1], [3, 3, 1]], [[4, 4, 1], [5, 5, 1], [6, 6, 1]], [[7, 7, 1], [8, 8, 1], [9, 9, 1]]]])
    init_b = np.array([[[[4, 1, 1], [2, 2, 1], [3, 3, 1]], [[4, 4, 1], [5, 5, 1], [6, 6, 1]], [[7, 7, 1], [8, 8, 1], [9, 9, 1]]]])
    
    feed_dict = {a: init_a, b: init_b}
    print(loss.eval(feed_dict))

0.333333
